# 开始构建一个对话机器人

## 构建机器人的个人属性

In [1]:
import xml.etree.ElementTree as et

template = et.parse('data/robot_template.xml')#加载xml

#测试一下
[[i.tag,i.text] for i in template.find('robot_info')]

[['name', 'Ash，中文名：小智'],
 ['sex', '你猜'],
 ['age', '刚满月'],
 ['marry', '未婚'],
 ['career', '程序员（客服）'],
 ['creator', '致敬豪哥'],
 ['mom', '瑶姐姐']]

In [2]:
import xml.etree.ElementTree as et
# from config import * #加载配置
from random import choice
import re

TEMPLATE_PATH = 'data/robot_template.xml'

class template():
    def __init__(self):
        self.template = et.parse(TEMPLATE_PATH)
        self.robot_info = self.load_robot_info()#加载个人属性
        self.temp = self.template.findall('temp')#加载问答样式

    def load_robot_info(self):
        rebot_info = self.template.find('robot_info')
        rebot_info_dict = {}
        for info in rebot_info:
            rebot_info_dict[info.tag] = info.text
        return rebot_info_dict
    
    def search_answer(self,question):
        keys,temp = False,None
        for i in self.temp:
            qs = i.find('question')
            for q in qs.findall('q'):
                if re.search(q.text,question):
                    keys = True
                    temp = i
                    break
            if keys:
                return choice([i.text for i in temp.find('answer').findall('a')]).format(**self.robot_info)
        return None

In [3]:
#测试一下
test = template()
while 1:
    q = input('请输入->')
    if q == ':':
        break
    print(test.search_answer(q))

请输入->你是谁
可以去问问苹果的Siri，我刚告诉她我叫Ash，中文名：小智
请输入->:


## 基于语料构建机器人

In [4]:
##预处理
#适当清洗语料+加载语料
from config import *
import re
import jieba
def load_seq_qa():
    q_list,a_list = [],[]
    with open(CORPUS_PATH,'r',encoding = 'utf-8') as f:
        for ind, i in enumerate(f):
            #print(f"ind:{ind},i{i}")
            i = jieba.lcut(i.strip())
            if ind % 2 == 0:
                q_list.append(i)#偶数行是答案
            else:
                a_list.append(i)
    return q_list,a_list

print("问题:\n",load_seq_qa()[0][:10],'\n\n',"答案:\n",load_seq_qa()[1][:10])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZHANG_~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.678 seconds.
Prefix dict has been built successfully.


问题:
 [['在', '吗'], ['在', '吗'], ['在', '不'], ['你好'], ['我', '叫'], ['我', '是'], ['你', '猜', '我', '是', '谁'], ['你', '猜', '我', '叫', '什么'], ['我', '肚子疼'], ['我', '头疼']] 

 答案:
 [['嗯', '，', '咋啦'], ['嗯', '，', '有事吗'], ['嗯', '呢', '，', '在', '呢', '~'], ['你好', '~', '，', '你', '是', '哪位'], ['你好', '呀', '~'], ['幸会幸', '会'], ['懒得', '猜'], ['太费', '内存', '，', '我', '不想', '猜'], ['多', '喝', '热水', '（', '如果', '你', '想', '查询', '相关', '疾病', '，', '请', '输入', '专有名词', '，', '例如', '：', '腹痛', '）'], ['用', '锤子', '砸', '一', '砸', '就', '好', '了', '（', '如果', '你', '想', '查询', '相关', '疾病', '，', '请', '输入', '专有名词', '，', '例如', '：', '头痛', '）']]


In [5]:
#针对提问，构建词库，将切割后的词转换为index，nlp常用手段
def build_vocab():
    q,_ = load_seq_qa()
    word_dict = set([j for i in q for j in i]) #set可以去重？
    print(word_dict) #所有question分词后放入一个set
    word_dict = dict(zip(word_dict,range(len(word_dict))))#不重复的所有的词映射到index
    return word_dict 
    
print(build_vocab())

{'关机', '好像', '小豪', '嘿嘿嘿', '游戏', '也', '魏振豪', '菜', '中午', '今天天气', '不', '知道', '咋样', '看', '同性恋', '嘎', '家人', 'HELLO', '嘴巴', '工作', '加油', '失望', '鼻子', '玩', '很', '伤心', '挺好吃', '娶', '这', '好', '男', '聊天', '没关系', '早餐', '猜', '发个', '我们', '不错', '幸福', '想', '调戏', '一', '动', '亲戚', '预报', '雨下', '夸夸', '没', '属', '你老', '电', '幽默', '再', '聊聊天', '、', '请客', '耶耶耶耶耶', '不好', '唱歌', '对', '33', '去死吧', '要', '骂人', '生气', '怎么样', '再见', '晚安', '踹', '厕所', '你好', '总吃电', '了', '傻', '为什么', '没有', '想干什么', '眼睛', '可爱', '电影', '傻子', '饿', '换', '内存', '第一次', '学得', '点哈', '会', '吧', '嗯', '我能', '是', '把', '哥哥', '不了', '肚子疼', '好开心', '还', '，', '？', '热', '头', '明天', '哈哈', '听说', '21', '过', '啥', '咱们', '男朋友', 'Hi', '真傻', '整理', '见面', '爱', '下去', '灰心', '你家', '不去', '好吃', '干', '洗澡', '机器人', '跟', '嘻嘻', '学习', '英文', '你', '星座', '爱情', '不是', '的', '红包', '呼呼', '1', '记住', '追', '在', '回复', '今天', '才', '睡觉', '呐', '嘻', '乘', '开心', '多次', '可以', '哪里', '愁', '发财', '走', '下午', '爸爸', '一直', '天气', '略略', '几', '无聊', '敢不敢', 'OK', '起来', '哈哈哈', '吃完饭', '气候', '现在', '真笨', '真大', '么', '表白', '脸', '

In [6]:
def build_word_embeding():
    q,_ = load_seq_qa()
    word_dict = build_vocab()
    word_embeding = {}
    for w in word_dict.keys():
        word_embeding[w] = []
    for ind,qs in enumerate(q):#qs及其对应的ind
        for w in qs: #w是词，qs是句子
            #print(f"w:{w}\n qs:{qs}")
            
            #把qs对应序号作为其中所有词w的的embeding，这个序号不同于word_dict中的序号
            word_embeding[w].append(ind)
    return word_embeding #字典元素 key是词, value是该词所属于的句子的index

build_word_embeding()

{'关机', '好像', '小豪', '嘿嘿嘿', '游戏', '也', '魏振豪', '菜', '中午', '今天天气', '不', '知道', '咋样', '看', '同性恋', '嘎', '家人', 'HELLO', '嘴巴', '工作', '加油', '失望', '鼻子', '玩', '很', '伤心', '挺好吃', '娶', '这', '好', '男', '聊天', '没关系', '早餐', '猜', '发个', '我们', '不错', '幸福', '想', '调戏', '一', '动', '亲戚', '预报', '雨下', '夸夸', '没', '属', '你老', '电', '幽默', '再', '聊聊天', '、', '请客', '耶耶耶耶耶', '不好', '唱歌', '对', '33', '去死吧', '要', '骂人', '生气', '怎么样', '再见', '晚安', '踹', '厕所', '你好', '总吃电', '了', '傻', '为什么', '没有', '想干什么', '眼睛', '可爱', '电影', '傻子', '饿', '换', '内存', '第一次', '学得', '点哈', '会', '吧', '嗯', '我能', '是', '把', '哥哥', '不了', '肚子疼', '好开心', '还', '，', '？', '热', '头', '明天', '哈哈', '听说', '21', '过', '啥', '咱们', '男朋友', 'Hi', '真傻', '整理', '见面', '爱', '下去', '灰心', '你家', '不去', '好吃', '干', '洗澡', '机器人', '跟', '嘻嘻', '学习', '英文', '你', '星座', '爱情', '不是', '的', '红包', '呼呼', '1', '记住', '追', '在', '回复', '今天', '才', '睡觉', '呐', '嘻', '乘', '开心', '多次', '可以', '哪里', '愁', '发财', '走', '下午', '爸爸', '一直', '天气', '略略', '几', '无聊', '敢不敢', 'OK', '起来', '哈哈哈', '吃完饭', '气候', '现在', '真笨', '真大', '么', '表白', '脸', '

{'关机': [14, 178],
 '好像': [78],
 '小豪': [21],
 '嘿嘿嘿': [126],
 '游戏': [208],
 '也': [38],
 '魏振豪': [19],
 '菜': [52],
 '中午': [59],
 '今天天气': [70, 71, 73],
 '不': [2, 46, 54, 68, 77, 82, 99, 137, 147],
 '知道': [77, 87],
 '咋样': [77],
 '看': [83],
 '同性恋': [103],
 '嘎': [125],
 '家人': [133, 135],
 'HELLO': [189],
 '嘴巴': [217],
 '工作': [39],
 '加油': [161],
 '失望': [154],
 '鼻子': [216],
 '玩': [207],
 '很': [22, 26, 97, 98, 101, 151, 154, 175],
 '伤心': [182],
 '挺好吃': [56],
 '娶': [119, 120],
 '这': [72],
 '好': [25, 70, 75, 116, 140, 141, 142, 157, 169, 175],
 '男': [81],
 '聊天': [139, 143, 200],
 '没关系': [180],
 '早餐': [63],
 '猜': [6, 7, 137, 149],
 '发个': [69],
 '我们': [23, 26],
 '不错': [71],
 '幸福': [145],
 '想': [57, 66, 105, 106, 119, 207, 208],
 '调戏': [168],
 '一': [231],
 '动': [31],
 '亲戚': [134],
 '预报': [78],
 '雨下': [76],
 '夸夸': [164, 166],
 '没': [47, 55, 105],
 '属': [209],
 '你老': [153],
 '电': [65],
 '幽默': [160],
 '再': [166],
 '聊聊天': [173],
 '、': [52],
 '请客': [84],
 '耶耶耶耶耶': [128],
 '不好': [73],
 '唱歌': [198, 199],
 '对

In [7]:
from collections import Counter
import numpy as np
import jieba

class corpusSearch():
    def __init__(self):
        self.q,self.a = load_seq_qa()
        self.word_embeding = build_word_embeding()
        self.threshold = 0.7

    def cosine_sim(self,a,b):#a:"我生气了" b:"我特别生气"
        a_count = Counter(a) #Counter({'我': 1, '生气': 1, '了': 1})
        b_count = Counter(b)#Counter({'我': 1, '特别': 1, '生气': 1})
        a_vec = []
        b_vec = []
        all_word = set(a+b) #{'生气', '我', '特别', '了'}
        #这边类似构建了一个句子的embeding
        for i in all_word:
            a_vec.append(a_count.get(i,0)) #[1, 1, 0,1] 特别没在a中出现
            b_vec.append(b_count.get(i,0)) #[1, 1, 1,0] 了没在b中出现
        #计算余弦相似
        a_vec = np.array(a_vec)
        b_vec = np.array(b_vec)
        cos = sum(a_vec*b_vec)/(sum(a_vec*a_vec)**0.5)/(sum(b_vec*b_vec)**0.5)#求两个向量的余弦相似度
        return round(cos,4)

    def search_answer(self,question):#找到相似度最高的句子
        search_list = []
        q_words = jieba.lcut(question)
        for q_word in q_words:
            index = self.word_embeding.get(q_word,list())#找出q_word对应的索引list
            search_list += index
        print(search_list) 
        if len(search_list) == 0:
            return None
        
        #挑选前3个出现概率最大的index作为句子的表征
        count_list = Counter(search_list)      #计算每个index出现的次数
        count_list = count_list.most_common(3) #挑选次数对多的前三个index作为句子的代表
        print(count_list)    #若question是"我特别生气"，则结果是:[(96, 2), (97, 2), (98, 2)]
        res_list = []
        for i,_ in count_list:
            q = self.q[i] 
            sim = self.cosine_sim(q,q_words)#分别求这三个index对应的语料库句子和所输入句子的相似度
            if sim > self.threshold:
                res_list.append([sim,self.a[i]])#找到对应q[i]的a[i],因为要解决的问题是根据问题检索答案
        res_list.sort()#排序从小到大排
        if len(res_list) == 0:
            return None
        else:
            return ''.join(res_list[-1][1])#最后一个代表sim最大，吧切分的词连成句子返回


In [8]:
test = corpusSearch()
while 1:
    question = input('请输入->') #比如输入:我很生气 找到对应答案：为什么?你犯病了吗？
    if question == ':':
        break
    print(test.search_answer(question))

{'关机', '好像', '小豪', '嘿嘿嘿', '游戏', '也', '魏振豪', '菜', '中午', '今天天气', '不', '知道', '咋样', '看', '同性恋', '嘎', '家人', 'HELLO', '嘴巴', '工作', '加油', '失望', '鼻子', '玩', '很', '伤心', '挺好吃', '娶', '这', '好', '男', '聊天', '没关系', '早餐', '猜', '发个', '我们', '不错', '幸福', '想', '调戏', '一', '动', '亲戚', '预报', '雨下', '夸夸', '没', '属', '你老', '电', '幽默', '再', '聊聊天', '、', '请客', '耶耶耶耶耶', '不好', '唱歌', '对', '33', '去死吧', '要', '骂人', '生气', '怎么样', '再见', '晚安', '踹', '厕所', '你好', '总吃电', '了', '傻', '为什么', '没有', '想干什么', '眼睛', '可爱', '电影', '傻子', '饿', '换', '内存', '第一次', '学得', '点哈', '会', '吧', '嗯', '我能', '是', '把', '哥哥', '不了', '肚子疼', '好开心', '还', '，', '？', '热', '头', '明天', '哈哈', '听说', '21', '过', '啥', '咱们', '男朋友', 'Hi', '真傻', '整理', '见面', '爱', '下去', '灰心', '你家', '不去', '好吃', '干', '洗澡', '机器人', '跟', '嘻嘻', '学习', '英文', '你', '星座', '爱情', '不是', '的', '红包', '呼呼', '1', '记住', '追', '在', '回复', '今天', '才', '睡觉', '呐', '嘻', '乘', '开心', '多次', '可以', '哪里', '愁', '发财', '走', '下午', '爸爸', '一直', '天气', '略略', '几', '无聊', '敢不敢', 'OK', '起来', '哈哈哈', '吃完饭', '气候', '现在', '真笨', '真大', '么', '表白', '脸', '

In [30]:
q,a =load_seq_qa()#确认下是不是
print(q[96:99])
print('\n')
print(a[96:99])
#可以看出输入"我生气" 根据相似度匹配到语料库中的question是"我生气了" ,该问题对应answer："那我也生气"，将这个答案作为输入句子的答案。

[['我', '生气', '了'], ['我', '很', '生气'], ['我', '现在', '很', '生气']]


[['那', '我', '也', '生气'], ['为什么', '？', '你', '犯病', '了', '？'], ['别生气', '，', '我们', '一起', '生', '孩子', '吧']]


## 基于知识图谱构建机器人（简单的关系应用）

In [9]:
from py2neo import Graph
from pyhanlp import *
from random import choice


#知识图谱的初步应用
class GraphSearch():
    def __init__(self):
        self.graph = Graph("http://localhost:7474", username="neo4j", password="123456")
        self.iswho_sql = "profile match p=(n)<-[r]-(b) where n.name='%s' return n.name,r.name,b.name"
        self.isrelation_sql = "profile match p=(n)<-[r]-(b) where n.name=~'.*%s' and b.name=~'.*%s' return n.name,r.name,b.name" 
    
    def search_answer(self,question):
        #使用HanLP进行词性判断
        sentence = HanLP.parseDependency(question)
        #后续可以替换成自己训练的模块首先针对句意进行分析，其次针对目标实体进行提取；但主要也是针对业务场景进行分析和处理
        seg = {}
        res_combine = ''
        for word in sentence.iterator(): 
        ##只处理nr名词：人，v动词，n名词，针对进行提问进行词性分析
            if word.POSTAG[0] == 'n' or word.POSTAG in ['v','r']:
                if word.POSTAG not in seg:
                    seg[word.POSTAG] = [word.LEMMA] #{'nr': ['邓小平']}
                else:
                    seg[word.POSTAG].append(word.LEMMA) 
        print(seg,'*'*10) 
        #简单基于词性和内容判断是否为目标句式'A是谁'以此使用知识图谱进行回答
        if 'v' in seg and '是' in seg['v']:
            if 'r' in seg and 'nr' in seg and '谁' in seg['r']:
                for person in seg['nr']:
                    res = self.graph.run(self.iswho_sql%(person)).data()
                    res_combine = []
                    for i in res[:10]:
                        res_combine.append('%s是:%s%s'%(i['n.name'],i['b.name'],i['r.name']))
                return choice(res_combine)
        #基于词性和内容判断是否为目标句式'A和B的关系'以此使用知识图谱进行回答
        if 'n' in seg and '关系' in seg['n']:
            if len(seg['nr']) == 2:
                print(seg,'*1'*10)
                res1 = self.graph.run(self.isrelation_sql%(seg['nr'][1],seg['nr'][0])).data()#从neo4j找关系
                if res1 != []:
                    res_combine = seg['nr'][0]+'的'+res1[0]['r.name']+'是'+seg['nr'][1]
                    return res_combine
                res2 = self.graph.run(self.isrelation_sql%(seg['nr'][0],seg['nr'][1])).data()
                if res2 != []:
                    res_combine = seg['nr'][1]+'的'+res2[0]['r.name']+'是'+seg['nr'][0]
                    return res_combine
        if res_combine == '':
            return None

In [10]:
test = GraphSearch()#要先打开neo4j
while 1:
    question = input('请输入->')
    if question == ':':
        break
    print(test.search_answer(question))

请输入->习近平和彭丽媛什么关系
{'nr': ['习近平', '彭丽媛'], 'r': ['什么'], 'n': ['关系']} **********
{'nr': ['习近平', '彭丽媛'], 'r': ['什么'], 'n': ['关系']} *1*1*1*1*1*1*1*1*1*1
彭丽媛的丈夫是习近平
请输入->:


## 基于基于网络数据进行回答

In [11]:
import requests
class InterNet():
    def __init__(self):
        pass
    def search_answer(self,question):
        url = 'https://api.ownthink.com/bot?appid=xiaosi&userid=user&spoken='
        try:
            text = requests.post(url+question).json()
            if 'message' in text and text['message'] == 'success':
                return text['data']['info']['text']
            else:
                return None
        except:
            return None

In [34]:
test = InterNet()
while 1:
    question = input('请输入->')
    if question == ':':
        break
    print(test.search_answer(question))

请输入->北京今天温度
别聊这个了，聊点有趣的怎么样？
请输入->北京天气
北京，今天晴，当前温度17°，空气质量优，东北风。明天多云，最低温度5°，最高温度15°，空气质量良。
请输入->:


# 服务化我的机器人

## 我的第一个flask服务

In [ ]:
'''
这里的代码在jupyter里运行不了
我的第一个flask_service.py
'''
from flask_cors import cross_origin
from flask import Flask,request,redirect,url_for
import requests,json
import time

#初始化一个flask
app = Flask(__name__)

@app.route('/test', methods=['GET', 'POST'])
@cross_origin()
def myfirst_service():
    if request.method ==  "POST":
        data = request.get_data().decode()
        data = json.loads(data)
        return json.dumps(data['question'],ensure_ascii=False)
    
@app.route('/test1', methods=['GET', 'POST'])
@cross_origin()
def myfirst_service1():
    if request.method ==  "POST":
        data = request.get_data().decode()
        data = json.loads(data)
        time.sleep(10)
        return json.dumps(data['question'],ensure_ascii=False)
    
@app.route('/', methods=['GET', 'POST'])
@cross_origin()
def myfirst_service2():
    return '人工智能我最爱'
    
    
if __name__ == "__main__":
    app.run(host='0.0.0.0',port=8080,threaded=True)

In [ ]:
'''
我的第一个send.py
'''
import requests
import json
url = 'http://127.0.0.1:8080/test'
question = '你好啊'
data = {
    'question':question
    }
print(requests.post(url,data=json.dumps(data)).json())

## 服务化各项我的机器人功能

    请大家自己尝试构建各项服务

## 测试各项服务

### 基于个人属性接口测试

In [13]:
#my_first_service.py服务要先运行起来
import requests
import json
while 1:
    question = input('请输入->')
    if question == ':':
        break
    data = {
        'question':question
        }
#     print('data是这个：',str(data))
    url = 'http://127.0.0.1:8080/template'
    answer = requests.post(url,data=json.dumps(data)).json()
    print(answer)

请输入->你是谁
可以去问问苹果的Siri，我刚告诉她我叫Ash，中文名：小智
请输入->你是男生吗
你觉得我是男的还是女的
请输入->:


### 基于语料搜索接口测试

In [14]:
while 1:
    question = input('请输入->')
    if question == ':':
        break
    data = {
        'question':question
        }
    url = 'http://127.0.0.1:8080/CorpusSearch'
    answer = requests.post(url,data=json.dumps(data)).json()
    print(answer)

请输入->我生气
那我也生气
请输入->:


### 基于知识图谱接口测试

In [15]:
while 1:
    question = input('请输入->')
    if question == ':':
        break
    data = {
        'question':question
        }
    url = 'http://127.0.0.1:8080/GraphSearch'
    answer = requests.post(url,data=json.dumps(data)).json()
    print(answer)

请输入->习近平和彭丽媛是什么关系
彭丽媛的丈夫是习近平
请输入->:


### 基于互联网接口测试

In [17]:
while 1:
    question = input('请输入->')
    if question == ':':
        break
    data = {
        'question':question
        }
    url = 'http://127.0.0.1:8080/InterNet'
    answer = requests.post(url,data=json.dumps(data)).json()
    print(answer)

请输入->北京气温
好冷，要抱抱。
请输入->杭州滨江气温
你喋喋不休的说这么多，我都不好意思不理你。
请输入->:


# 课堂练习

Q1：

    如何在后台实现各项机器人功能的自动流转？
    
Q2：

    如何更加智能化实现流转？
    
Q3：

    还有哪些可以优化？

## 串行实现功能流转

In [18]:
import requests
import json

# data = {
#    'test':'你好啊我的第一次服务'
#   }

# url = 'http://127.0.0.1:8080/test'
# r = requests.post(url,data=json.dumps(data))
# print(r.json())

while 1:
    question = input('请输入->')
    if question == ':':
        break
    data = {
    'question':question
    }
    url = 'http://127.0.0.1:8080/template'
    answer = requests.post(url,data=json.dumps(data)).json()
    if answer is not None:
        print(answer)
        continue
    url = 'http://127.0.0.1:8080/CorpusSearch'
    answer = requests.post(url,data=json.dumps(data)).json()
    if answer is not None:
        print(answer)
        continue
    print('请稍等哦~我正在尽力搜索我的记忆ing~')
    url = 'http://127.0.0.1:8080/GraphSearch'
    answer = requests.post(url,data=json.dumps(data)).json()
    if answer is not None:
        print(answer)
        continue
    print('小智发现我的记忆力缺少了点内容，o(╥﹏╥)o，让我查查网上怎么说~')
    url = 'http://127.0.0.1:8080/InterNet'
    answer = requests.post(url,data=json.dumps(data)).json()
    print(answer)


请输入->你是谁
我叫 Ash，中文名：小智
请输入->毛泽东是谁
请稍等哦~我正在尽力搜索我的记忆ing~
毛泽东是:柯蓝[中国内地演员]主要作品
请输入->习近平和彭丽媛什么关系
请稍等哦~我正在尽力搜索我的记忆ing~
彭丽媛的丈夫是习近平
请输入->:


## 基于多线程实现流转

In [6]:
import time
import requests
import json
import _thread

question,answer = [None,None,None,None],[None,None,None,None]

def test_template():
    global question,answer
    while 1:
        if question[0] is not None:
            data = {
                'question':question[0]
            }
            url = 'http://127.0.0.1:8080/template'
            res = requests.post(url,data=json.dumps(data)).json()
            if question[0] is not None:
                answer[0] = res
                question[0] = None
            time.sleep(1)

def test_CorpusSearch():
    global question,answer
    while 1:
        if question[1] is not None:
            data = {
                'question':question[1]
            }
            url = 'http://127.0.0.1:8080/CorpusSearch'
            res = requests.post(url,data=json.dumps(data)).json()
            if question[1] is not None:
                answer[1] = res
                question[1] = None
            time.sleep(1)

def test_GraphSearch():
    global question,answer
    while 1:
        if question[2] is not None:
            data = {
                'question':question[2]
            }
            url = 'http://127.0.0.1:8080/GraphSearch'
            res = requests.post(url,data=json.dumps(data)).json()
            if question[2] is not None:
                answer[2] = res
                question[2] = None
            time.sleep(1)

def test_InterNet():
    global question,answer
    while 1:
        if question[3] is not None:
            data = {
                'question':question[3]
            }
            url = 'http://127.0.0.1:8080/InterNet'
            res = requests.post(url,data=json.dumps(data)).json()
            if question[3] is not None:
                answer[3] = res
                question[3] = None
            time.sleep(1)


from mychatbot import *
import xml.etree.ElementTree as et
from config import *

def get_default():
    answers = []
    data = et.parse(TEMPLATE_PATH)
    a = data.find('default').findall('a')
    for i in a:
        answers.append(i.text)
    return answers

if __name__ == '__main__':
    default_answer = get_default()
    _thread.start_new_thread(test_template,())
    _thread.start_new_thread(test_CorpusSearch,())
    _thread.start_new_thread(test_GraphSearch,())
    _thread.start_new_thread(test_InterNet,())
    while 1:
        answer  = [None,None,None,None]
        question = [None,None,None,None]
        get = input('请输入->')
        if get == ':':
            break
        if get != '':
            question = [get,get,get,get]
        if question[0] is not None:
            n = 0
            while 1:
                k = False
                for i in answer:
                    if i is not None:
                        k = True
                        print(i)
                        break
                if k:
                    break
                time.sleep(1)
                n += 1
                if n > 10:
                    print(choice(default_answer))
                    break

                

请输入->多线程是什么意思
能跟我分享你的喜悦吗？我很期待呐~
请输入->:


## 使用意图识别

    等待后面我们的课程补充